In [ ]:
import os
import json
import pandas as pd

base_path = "."
target_keys = [
    "timestamp",
    "model_purpose",
    "predicts",
    "special_features",
    "f1",
    "recall",
    "precision",
    "bal_accuracy",
    "roc_auc_score",
    "classification_report__no dia__f1-score",
    "classification_report__no dia__recall",
    "classification_report__dia__f1-score",
    "classification_report__dia__recall",
    "classification_report__pre__f1-score",
]

subdirs = [
    d for d in os.listdir(base_path) if os.path.isdir(d) and not d.startswith(".")
]


rows = []

for d in subdirs:
    result_path = os.path.join(base_path, d, f"{d}.results.json")
    row = {}
    if os.path.exists(result_path):
        with open(result_path, "r") as f:
            data = json.load(f)

        for key in target_keys:
            val = data
            for part in key.split("__"):
                if isinstance(val, dict) and part in val:
                    val = val[part]
                else:
                    val = None
                    break
            row[key] = val
    else:
        for key in target_keys:
            row[key] = None
    row["dir"] = d
    rows.append(row)


df = pd.DataFrame(rows)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)

for col in [c for c in df.columns if c.startswith("classification_report")]:
    df.rename(
        columns={
            col: col.replace("classification_report__", "cl_rep__").replace(" ", "_")
        },
        inplace=True,
    )

df.sort_values(by="f1", ascending=False, inplace=True)
display(df)


In [ ]:
# df as markdown table
from tabulate import tabulate
print(tabulate(df.reset_index(), headers="keys", tablefmt="github", showindex=False))

In [ ]:
import matplotlib.pyplot as plt

# Select only numeric columns
num_cols = df.select_dtypes(include="number").columns


# Fallback for row labels
def get_row_label(row):
    name = row.name.strftime("%m-%d %H:%M:%S") + " "

    if pd.notna(row.get("special_features")) and row["special_features"] != "":
        name += str(row["special_features"])
    else:
        name += str(row.get("model_purpose", ""))

    pred_lab = row["predicts"]

    name = f"{name} ({', '.join(pred_lab)})"
    return name

df.sort_values(by="timestamp", ascending=True, inplace=True)

# Create figure and axis
fig, ax = plt.subplots(figsize=(12, 6))
lines = []
alpha = 1

# Plot each row
for idx, row in df.iterrows():
    label = get_row_label(row)
    (line,) = ax.plot(num_cols, row[num_cols], marker="o", label=label, alpha=alpha)
    lines.append((idx, line, row))

# Mark maximum value per column
for col in num_cols:
    max_idx = df[col].idxmax()
    row = df.loc[max_idx]
    y = row[col] + 0.05
    x = list(num_cols).index(col)

    # Get the color of the corresponding line
    line_color = next((l.get_color() for i, l, r in lines if i == max_idx), "black")
    ax.plot(x, y, marker="v", color=line_color, markersize=8, alpha=alpha)

# Axis and legend settings
ax.set_xticks(range(len(num_cols)))
ax.set_xticklabels(num_cols, rotation=10)
ax.legend(bbox_to_anchor=(0.5, -0.11), loc="upper center", ncol=min(3, len(df)))
ax.set_title("Comparison of models by selected metrics")
ax.grid(axis="y", linestyle="--", linewidth=0.5)

plt.tight_layout()
plt.show()
